In [ ]:
%pip install pandas pyarrow
%pip install tqdm
%pip install plotly
%pip install nbformat
%pip install ipywidgets
%pip install numba

In [ ]:
# function to download data.

import requests
from tqdm import tqdm

def get_filename(ticker, interval, year):
    return f'data/{ticker}_{interval}_{year}.parquet'

def download(ticker, interval, year):
    filename = get_filename(ticker, interval, year)
    # create directory if it doesn't exist
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    if year:
      # url for seconds
      url = f"https://barsv.com/api/download/file/{ticker}/{year}?format=parquet&interval={interval}"
    else:
      # url for minutes+
      url = f"https://barsv.com/api/download/file/{ticker}/{ticker}_{interval}?interval={interval}&format=parquet"
    response = requests.get(url, stream=True)

    # Get the total file size from the content-length header
    total_size = int(response.headers.get('content-length', 0))

    with open(filename, 'wb') as f, tqdm(
        desc=filename,
        total=total_size,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for data in response.iter_content(chunk_size=1024):
            size = f.write(data)
            bar.update(size)

In [ ]:
ticker = 'AAPL'
interval = '5s'
year = '2024'

In [ ]:
import os

filename = get_filename(ticker, interval, year)
# if file doesn't exist then download it.
if not os.path.exists(filename):
	download(ticker, interval, year)

In [ ]:
# plot last points.
import pandas as pd
import plotly.express as px

df = pd.read_parquet(filename)

fig = px.line(df[-1000:], y='open', title=f'{ticker} Open Prices')
fig.show()

In [ ]:
total_bars = df.shape[0]
print(f'Total bars: {total_bars} ({total_bars:,})')

In [ ]:
# Set trailing stop-loos levels
import numpy as np

stop_loss_percents = np.concatenate((np.arange(-0.4, 0, 0.04), np.arange(0.04, 0.44, 0.04)))
stop_loss_levels = stop_loss_percents / 100

print("Уровни стоп-лосса (в процентах):")
print(stop_loss_percents)
print("\nУровни стоп-лосса (в долях):")
print(stop_loss_levels)

In [ ]:
import numba

@numba.jit(nopython=True)
def calculate_pl_numba(df_open, df_low, df_high, df_close, stop_loss_levels):
    """
    Оптимизированная с помощью Numba функция для расчета P/L со скользящим стоп-лоссом.
    """
    num_entries = len(df_open)
    num_sl_levels = len(stop_loss_levels)
    # Numba плохо работает с list-of-dicts, поэтому будем собирать результаты в NumPy массив
    results_np = np.empty(( (num_entries - 1) * num_sl_levels, 4)) # index, sl_level, profit, entry_price
    result_idx = 0

    # Итерация по каждой точке в экспериментальном датафрейме (кроме последней)
    for i in range(num_entries - 1):
        entry_price = df_open[i]

        # Итерация по каждому уровню стоп-лосса
        for sl_idx in range(num_sl_levels):
            sl_level = stop_loss_levels[sl_idx]
            if sl_level == 0:
                continue

            stop_price = entry_price * (1 + sl_level)
            exit_price = 0.0
            trade_closed = False

            # Поиск момента срабатывания стоп-лосса в будущем
            for j in range(i + 1, num_entries):
                future_low = df_low[j]
                future_high = df_high[j]

                # Для длинных позиций (sl_level < 0)
                if sl_level < 0:
                    if future_low <= stop_price:
                        exit_price = stop_price
                        trade_closed = True
                        break
                    else:
                        new_stop_price = future_high * (1 + sl_level)
                        if new_stop_price > stop_price:
                            stop_price = new_stop_price
                # Для коротких позиций (sl_level > 0)
                elif sl_level > 0:
                    if future_high >= stop_price:
                        exit_price = stop_price
                        trade_closed = True
                        break
                    else:
                        new_stop_price = future_low * (1 + sl_level)
                        if new_stop_price < stop_price:
                            stop_price = new_stop_price

            if not trade_closed:
                exit_price = df_close[-1]

            profit = 0.0
            if sl_level < 0:
                profit = 100 * (exit_price - entry_price) / entry_price
            elif sl_level > 0:
                profit = 100 * (entry_price - exit_price) / entry_price

            results_np[result_idx, 0] = i
            results_np[result_idx, 1] = sl_level
            results_np[result_idx, 2] = profit
            results_np[result_idx, 3] = entry_price

            result_idx += 1

    return results_np[:result_idx] # Возвращаем только заполненную часть массива

def calculate_pl(df_exp):
    # Подготовим данные для Numba (она лучше работает с NumPy-массивами, чем с Pandas Series)
    df_open_np = df_exp['open'].to_numpy()
    df_low_np = df_exp['low'].to_numpy()
    df_high_np = df_exp['high'].to_numpy()
    df_close_np = df_exp['close'].to_numpy()

    numba_results_np = calculate_pl_numba(df_open_np, df_low_np, df_high_np, df_close_np, stop_loss_levels)

    # Преобразуем результат обратно в DataFrame
    results_df = pd.DataFrame(numba_results_np, columns=['index', 'sl_level', 'profit', 'entry_price'])
    # Конвертируем 'index' в int для корректного использования в качестве индекса
    results_df['index'] = results_df['index'].astype(int)
    return results_df


In [ ]:
# Шаг 4: Визуализация результатов
import plotly.graph_objects as go

def plot_sl_profits(df_exp, results_df):
    fig = go.Figure()

    # Добавляем точки стоп-лоссов
    # Для лучшей производительности мы можем добавить все точки одним вызовом
    # Отфильтруем нулевые SL, так как они не несут информации
    plot_df = results_df#[results_df['sl_level'] != 0].copy()
    plot_df['sl_price'] = plot_df['entry_price'] * (1 + plot_df['sl_level'])

    min_profit = plot_df['profit'].min()
    max_profit = plot_df['profit'].max()

    # Calculate normalized position for zero (between 0 and 1)
    zero_position = (0 - min_profit) / (max_profit - min_profit)

    custom_colorscale = [
        [0.0, 'red'],           # Minimum profit = red
        [zero_position, 'white'], # Zero profit = white
        [1.0, 'blue']           # Maximum profit = blue
    ]

    fig.add_trace(go.Scatter(
        x=plot_df['index'],
        y=plot_df['sl_price'],
        mode='markers',
        marker=dict(
            size=3,
            color=plot_df['profit'],
            colorscale=custom_colorscale,
            showscale=True,
            colorbar=dict(title='Profit (%)'),
            cmin=min_profit,  # Set the actual data range
            cmax=max_profit
        )
    ))

    # add ohl prices on top
    fig.add_trace(go.Scatter(
        #x=df_exp['timestamp'],
        y=df_exp['open'],
        mode='lines',
        line=dict(color='black', width=1)
    ))
    fig.add_trace(go.Scatter(
        y=df_exp['high'],
        mode='lines',
        line=dict(color='black', width=1)
    ))
    fig.add_trace(go.Scatter(
        y=df_exp['low'],
        mode='lines',
        line=dict(color='black', width=1)
    ))

    fig.update_layout(
        title="Trailing SL profits",
        showlegend=False
    )

    fig.show()

In [ ]:
shift = -40 * 1000
window = 10 * 1000
df_exp = df[shift:shift + window].copy()
results_df = calculate_pl(df_exp)

In [ ]:
plot_sl_profits(df_exp, results_df)

In [ ]:
shift = -50 * 1000
window = 10 * 1000
df_exp = df[shift:shift + window].copy()
results_df = calculate_pl(df_exp)

In [ ]:
plot_sl_profits(df_exp, results_df)

In [ ]:
shift = -45 * 1000
window = 15 * 1000
df_exp = df[shift:shift + window].copy()
results_df = calculate_pl(df_exp)

In [ ]:
plot_sl_profits(df_exp, results_df)

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Custom plot function showing specified percentiles and the mean for each category
def box(df, x, y, title=None, percentiles=[0, 10, 25, 50, 75, 90, 100]):
    """
    Custom box-like plot function showing specified percentiles and the average for each category.
    
    Parameters:
    df         : pandas.DataFrame
    x          : str, column name for categories
    y          : str, column name for values
    title      : str, plot title
    percentiles: list of ints, percentiles to display (e.g., [0,10,25,50,75,90,100])
    """
    # Ensure percentiles are sorted and converted to fractions
    percentiles_sorted = sorted(percentiles)
    quantile_fracs = [p / 100 for p in percentiles_sorted]

    # Compute the specified percentile values for each group
    grouped_q = df.groupby(x)[y].quantile(quantile_fracs).unstack()
    # Compute the mean value for each group
    grouped_mean = df.groupby(x)[y].mean()

    # Initialize figure
    fig = go.Figure()

    # Add a trace for each percentile
    for p, frac in zip(percentiles_sorted, quantile_fracs):
        fig.add_trace(go.Scatter(
            x=grouped_q.index,
            y=grouped_q[frac],
            mode='lines+markers',
            name=f'{p}th percentile',
            line=dict(dash='solid' if p == 50 else 'dash'),
            marker=dict(size=6)
        ))

    # Add mean as a bold black line
    fig.add_trace(go.Scatter(
        x=grouped_mean.index,
        y=grouped_mean.values,
        mode='lines',
        name='Mean',
        line=dict(color='black', width=4),
        hovertemplate='Mean: %{y:.2f}<extra></extra>'
    ))

    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title=x,
        yaxis_title=y,
        legend_title='Metrics',
        template='plotly_white'
    )

    return fig

# Example usage:
# fig = box(results_df, x='sl_level', y='profit',
#           title='Profit Distribution by Stop-Loss Level',
#           percentiles=[0, 10, 25, 50, 75, 90, 100])
# fig.show()


In [ ]:
# Example usage
fig = box(results_df, x='sl_level', y='profit',
    title='Profit Distribution by Stop-Loss Level',
    percentiles=[0, 25, 50, 75, 90, 100])
fig.show()
